In [1]:
import os
import cv2 as cv
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

In [2]:
data = []
labels = []

In [3]:
imgSize = 300
folder_data = r'D:\Vietname_sig\Data'
label_to_int = {'Hello': 0, 'Thanks': 1, 'ByeBye': 2, 'SeeYouAgain': 3, 'Sorry': 4}

In [4]:
for foldername in os.listdir(folder_data):
    if not foldername.startswith('.'):
        folder_path = os.path.join(folder_data, foldername)
        for filename in os.listdir(folder_path):
            if not filename.startswith('.'):
                img = cv.imread(os.path.join(folder_path, filename))
                img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
                img = cv.resize(img, (imgSize, imgSize))
                data.append(img)
                labels.append(foldername)

In [5]:
data = np.array(data, dtype="float32") / 255.0
labels = [label_to_int[label] for label in labels]
labels = np.array(labels)

In [6]:
labels = to_categorical(labels)

In [7]:
trainX, tempX, trainY, tempY = train_test_split(data, labels, test_size=0.2, random_state=2)
valX, testX, valY, testY = train_test_split(tempX, tempY, test_size=0.5, random_state=2)

In [8]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation="relu", input_shape=(imgSize, imgSize, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(5, activation="softmax"))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 35, 35, 128)      0

In [10]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [11]:
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(trainX.reshape(-1, imgSize, imgSize, 1))

In [12]:
model.fit(datagen.flow(trainX.reshape(-1, imgSize, imgSize, 1), trainY, batch_size=32),
          validation_data=(valX.reshape(-1, imgSize, imgSize, 1), valY),
          epochs= 25, steps_per_epoch=len(trainX) // 32)

Epoch 1/25
26/26 [==============================] - 9s 191ms/step - loss: 2.1718 - accuracy: 0.2430 - val_loss: 1.5658 - val_accuracy: 0.3585
Epoch 2/25
26/26 [==============================] - 4s 133ms/step - loss: 1.5428 - accuracy: 0.3028 - val_loss: 1.2061 - val_accuracy: 0.5472
Epoch 3/25
26/26 [==============================] - 4s 130ms/step - loss: 1.4157 - accuracy: 0.3980 - val_loss: 1.1105 - val_accuracy: 0.5283
Epoch 4/25
26/26 [==============================] - 4s 131ms/step - loss: 1.2181 - accuracy: 0.4933 - val_loss: 0.8714 - val_accuracy: 0.7925
Epoch 5/25
26/26 [==============================] - 4s 132ms/step - loss: 1.0558 - accuracy: 0.5922 - val_loss: 0.5003 - val_accuracy: 0.9434
Epoch 6/25
26/26 [==============================] - 4s 134ms/step - loss: 0.9860 - accuracy: 0.6264 - val_loss: 0.3591 - val_accuracy: 0.9434
Epoch 7/25
26/26 [==============================] - 4s 132ms/step - loss: 0.8084 - accuracy: 0.6984 - val_loss: 0.2367 - val_accuracy: 0.9717
Epoch 

In [13]:
model.save('Model.h5')